In [127]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import ast

from fastDamerauLevenshtein import damerauLevenshtein
from opportunistic_planning import data, sequence
from imp import reload
from scipy.stats import friedmanchisquare, wilcoxon
reload(data)
reload(sequence)

<module 'opportunistic_planning.sequence' from '/home/zauri/files/anaconda3/lib/python3.7/site-packages/opportunistic_planning/sequence.py'>

In [128]:
# read data

df = pd.read_csv('all_task_environments_2021-01-26.csv', header=0)

In [129]:
# dict generation ~1 sec

%time distances_dict = data.generate_distances_dict(df)

CPU times: user 1.05 s, sys: 44.2 ms, total: 1.09 s
Wall time: 1.06 s


In [142]:
# n=1: ~4min
# n=10: ~15min
# n050: ~1h
# n=100: 2h 4min
%time results_new = data.calculate_edit_distances_prequential(df, distances_dict=distances_dict, n=50)

CPU times: user 1h 5min 26s, sys: 1.01 s, total: 1h 5min 27s
Wall time: 1h 5min 26s


In [143]:
results_new

,"c: 1.0; k: 0.0,0.1,1.1; x","c: 1.0; k: 0.0,0.1,1.1; y","c: 1.0; k: 0.0,0.1,1.1; z","c: 1.0; k: 0.0,0.1,1.1; xy","c: 1.0; k: 0.0,0.1,1.1; xz","c: 1.0; k: 0.0,0.1,1.1; yz","c: 1.0; k: 0.0,0.1,1.1; xyz","c: 1.1; k: 0.0,0.1,1.1; x","c: 1.1; k: 0.0,0.1,1.1; y","c: 1.1; k: 0.0,0.1,1.1; z",...,"c: 1.9; k: 0.8,0.9,1.9; x","c: 1.9; k: 0.8,0.9,1.9; y","c: 1.9; k: 0.8,0.9,1.9; z","c: 1.9; k: 0.8,0.9,1.9; xy","c: 1.9; k: 0.8,0.9,1.9; xz","c: 1.9; k: 0.8,0.9,1.9; yz","c: 1.9; k: 0.8,0.9,1.9; xyz",sequence,error,ID
0,0.216667,0.116667,0.216667,0.116667,0.216667,0.116667,0.116667,0.216667,0.116667,0.216667,...,0.216667,0.216667,0.216667,0.216667,0.216667,0.216667,0.216667,pocgkr,0.723,a1
1,0.227551,0.171429,0.183333,0.191837,0.227551,0.171429,0.191837,0.227551,0.171429,0.183333,...,0.227551,0.207143,0.147619,0.227551,0.227551,0.207143,0.227551,cgwpcfks,0.785,a3
2,0.136607,0.196875,0.075149,0.181250,0.136607,0.196875,0.181250,0.136607,0.196875,0.075149,...,0.118750,0.151042,0.196875,0.118750,0.118750,0.151042,0.118750,kfsfkspwg,0.806,a5
3,0.120238,0.151488,0.111279,0.151488,0.120238,0.151488,0.151488,0.120238,0.151488,0.111279,...,0.120238,0.087500,0.111279,0.120238,0.120238,0.087500,0.120238,pfkswkfsococg,0.862,a11
4,0.181250,0.157440,0.176042,0.157440,0.181250,0.157440,0.157440,0.181250,0.157440,0.176042,...,0.181250,0.135417,0.056250,0.181250,0.181250,0.135417,0.181250,wptgkfsoc,0.806,a13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,0.190000,0.156667,0.116667,0.123333,0.156667,0.156667,0.090000,0.198333,0.156667,0.116667,...,0.190000,0.156667,0.116667,0.123333,0.090000,0.156667,0.090000,hhsdbg,0.723,v8
176,0.133333,0.116667,0.116667,0.050000,0.116667,0.116667,0.050000,0.166667,0.116667,0.116667,...,0.150000,0.116667,0.116667,0.050000,0.050000,0.116667,0.050000,hhsdgb,0.723,v9
177,0.150000,0.066667,0.106667,0.066667,0.000000,0.106667,0.066667,0.166667,0.066667,0.106667,...,0.166667,0.106667,0.106667,0.000000,0.066667,0.106667,0.066667,hhsgdb,0.723,v10
178,0.150000,0.106667,0.106667,0.066667,0.000000,0.106667,0.033333,0.166667,0.086667,0.106667,...,0.150000,0.106667,0.106667,0.000000,0.033333,0.106667,0.066667,hhsgdb,0.723,v11


In [172]:
#results_new.T.to_csv('results/results_all_n50_2021-01-26.csv', header=True, index=True)

In [171]:
lowest, lowest_idx, lowest_median, list_mean, results_mean = data.get_lowest_error(results_new)
lowest, lowest_idx, lowest_median

(0.13559444427721137,
 Index(['c: 1.8; k: 0.2,0.3,1.6; xyz'], dtype='object'),
 0.13527777777777777)

### Plot dimensions

In [145]:
# Define input for plots

c = [float(x[3:6]) for x in results_new.columns.tolist() if x != 'sequence' and x != 'ID' and x != 'error']
k = [float(x[11:14]) for x in results_new.columns.tolist() if x != 'sequence' and x != 'ID' and x != 'error']
dim = [x.strip() for x in results_new.columns[:-3].str.split(';').str[2]]
median = [x for x in results_mean.loc['mean'].tolist() if pd.notnull(x)]

In [146]:
%matplotlib qt

#cm = ['red','blue','green','magenta','cyan','orange','grey']
cm = ['#e74c3c', '#0064a3', '#70b85d', '#287d78', '#54d0ff', '#f1e664', '#fd8f00']
dim_num = [0 if x=='x' else 1 if x=='y' else 2 if x=='z' else 3 if x=='xy' else 4 if x=='xz' else 5 if x=='yz' else 6 for x in dim]

cmap = matplotlib.colors.ListedColormap(cm)

ticks = ['x', 'y', 'z', 'xy', 'xz', 'yz', 'xyz']
norm = matplotlib.colors.BoundaryNorm(ticks, cmap.N)

# create figure, 3d grid, set background to white
fig2 = plt.figure(figsize=(12,8))
ax2 = fig2.add_subplot(111, projection='3d')
ax2.w_xaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax2.w_yaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax2.w_zaxis.set_pane_color((1.0,1.0,1.0,1.0))

# create plot w/ median edit distance
img = ax2.scatter(c, k, median, alpha=0.5, s=38, c=dim_num, cmap=cmap)

# plot horizontal plane for baseline
xx, yy = np.meshgrid(np.linspace(1,1.9), np.linspace(0,0.9))
#yy = np.meshgrid(range(2), range(2))
zz = xx * 0 + np.mean(rnn)
ax2.plot_surface(xx, yy, zz, alpha=0.5)

# set labels
ax2.set_ylabel('parameter k', fontsize=20, labelpad=10)
ax2.set_xlabel('parameter c', fontsize=20, labelpad=10)
ax2.set_zlabel('normalized edit distance', fontsize=20, labelpad=7)
#plt.title('Average edit distance (1 step)', fontsize=24)

# create colorbar
cb = plt.colorbar(img, cax = fig2.add_axes([0.82,0.3,0.03,0.4]))
cb.set_alpha(0.8)
cb.draw_all()
cb.ax.set_yticklabels(ticks, fontsize=16)
plt.show()

#plt.gca().xaxis.set_major_locator(plt.NullLocator())
#plt.gca().yaxis.set_major_locator(plt.NullLocator())
#plt.savefig('figures/test.svg', format='svg', bbox_inches='tight', pad_inches = 0)

/home/zauri/files/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: FixedFormatter should only be used together with FixedLocator


In [147]:
cpt = [0.25, 0.16666666666666663, 0.13392857142857145, 0.10555555555555557, 0.15476190476190477, 0.26666666666666666, 0.25, 0.22499999999999998, 0.125, 0.1833333333333333, 0.1428571428571429, 0.1428571428571429, 0.11805555555555558, 0.25, 0.25, 0.25, 0.15476190476190477, 0.25, 0.11111111111111116, 0.09545454545454546, 0.13392857142857145, 0.19999999999999996, 0.11249999999999999, 0.09545454545454546, 0.19999999999999996, 0.16666666666666663, 0.11111111111111116, 0.1833333333333333, 0.22499999999999998, 0.26666666666666666, 0.1428571428571429, 0.10555555555555557, 0.14583333333333331, 0.16666666666666663, 0.22499999999999998, 0.1428571428571429, 0.14583333333333331, 0.19999999999999996, 0.14583333333333331, 0.25, 0.16666666666666663, 0.2916666666666667, 0.1428571428571429, 0.20833333333333331, 0.16666666666666663, 0.25, 0.26666666666666666, 0.15476190476190477, 0.19999999999999996, 0.16666666666666663, 0.25, 0.22499999999999998, 0.10555555555555557, 0.22499999999999998, 0.19999999999999996, 0.14583333333333331, 0.1833333333333333, 0.22499999999999998, 0.1428571428571429, 0.2916666666666667, 0.33333333333333337, 0.13392857142857145, 0.5, 0.22499999999999998, 0.19999999999999996, 0.26666666666666666, 0.10555555555555557, 0.13392857142857145, 0.19999999999999996, 0.15476190476190477, 0.19999999999999996, 0.25, 0.1428571428571429, 0.16666666666666663, 0.14583333333333331, 0.13392857142857145, 0.13392857142857145, 0.22499999999999998, 0.12698412698412703, 0.16666666666666663, 0.2916666666666667, 0.2916666666666667, 0.2916666666666667, 0.2916666666666667, 0.4166666666666667, 0.2916666666666667, 0.33333333333333337, 0.25, 0.4166666666666667, 0.4166666666666667, 0.2916666666666667, 0.25, 0.33333333333333337, 0.1833333333333333, 0.25, 0.1428571428571429, 0.2916666666666667, 0.5, 0.19999999999999996, 0.2916666666666667, 0.16666666666666663, 0.16666666666666663, 0.19999999999999996, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.19999999999999996, 0.16666666666666663, 0.16666666666666663, 0.19999999999999996, 0.16666666666666663, 0.19999999999999996, 0.19999999999999996, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.16666666666666663, 0.19999999999999996, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.19999999999999996, 0.16666666666666663, 0.19999999999999996, 0.19999999999999996, 0.16666666666666663, 0.19999999999999996, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.19999999999999996, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.22499999999999998, 0.20833333333333331, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996]

In [148]:
rnn = [0.19999999999999996, 0.1428571428571429, 0.11805555555555558, 0.11805555555555558, 0.13392857142857145, 0.09999999999999998, 0.25, 0.22499999999999998, 0.16666666666666663, 0.05555555555555558, 0.125, 0.16666666666666663, 0.11805555555555558, 0.16666666666666663, 0.25, 0.25, 0.11249999999999999, 0.16666666666666663, 0.125, 0.08012820512820512, 0.10555555555555557, 0.1428571428571429, 0.09545454545454546, 0.0839160839160839, 0.16666666666666663, 0.1428571428571429, 0.09999999999999998, 0.17142857142857143, 0.22499999999999998, 0.09999999999999998, 0.1428571428571429, 0.11805555555555558, 0.15476190476190477, 0.1428571428571429, 0.1833333333333333, 0.1428571428571429, 0.15476190476190477, 0.16666666666666663, 0.13392857142857145, 0.0, 0.25, 0.26666666666666666, 0.16666666666666663, 0.15476190476190477, 0.125, 0.33333333333333337, 0.26666666666666666, 0.13392857142857145, 0.19999999999999996, 0.19999999999999996, 0.16666666666666663, 0.09999999999999998, 0.10555555555555557, 0.0, 0.16666666666666663, 0.15476190476190477, 0.13392857142857145, 0.15476190476190477, 0.125, 0.2916666666666667, 0.0, 0.10101010101010105, 0.5, 0.20833333333333331, 0.125, 0.09999999999999998, 0.09545454545454546, 0.09545454545454546, 0.19999999999999996, 0.1833333333333333, 0.19999999999999996, 0.33333333333333337, 0.09999999999999998, 0.1428571428571429, 0.11805555555555558, 0.038461538461538436, 0.13392857142857145, 0.22499999999999998, 0.09545454545454546, 0.1428571428571429, 0.26666666666666666, 0.2916666666666667, 0.26666666666666666, 0.2916666666666667, 0.4166666666666667, 0.26666666666666666, 0.33333333333333337, 0.33333333333333337, 0.4166666666666667, 0.4166666666666667, 0.2916666666666667, 0.16666666666666663, 0.33333333333333337, 0.22499999999999998, 0.25, 0.125, 0.2916666666666667, 0.5, 0.25, 0.2916666666666667, 0.16666666666666663, 0.19999999999999996, 0.19999999999999996, 0.16666666666666663, 0.1428571428571429, 0.0, 0.19999999999999996, 0.0, 0.16666666666666663, 0.125, 0.19999999999999996, 0.16666666666666663, 0.0, 0.0, 0.19999999999999996, 0.0, 0.125, 0.19999999999999996, 0.16666666666666663, 0.1428571428571429, 0.0, 0.16666666666666663, 0.0, 0.19999999999999996, 0.19999999999999996, 0.1428571428571429, 0.16666666666666663, 0.16666666666666663, 0.1428571428571429, 0.0, 0.1428571428571429, 0.1428571428571429, 0.16666666666666663, 0.19999999999999996, 0.16666666666666663, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.0, 0.16666666666666663, 0.0, 0.19999999999999996, 0.0, 0.0, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.1428571428571429, 0.19999999999999996, 0.1428571428571429, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.125, 0.1428571428571429, 0.0, 0.0, 0.16666666666666663, 0.19999999999999996, 0.19999999999999996, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.1428571428571429, 0.16666666666666663, 0.19999999999999996, 0.1428571428571429, 0.2916666666666667, 0.08333333333333331, 0.25, 0.0, 0.19999999999999996, 0.0, 0.25, 0.0, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.25]

In [149]:
len(cpt), len(rnn)

(180, 180)

### Plot best model with baselines

In [181]:
#%matplotlib inline
%matplotlib qt

IDs = df['ID']

error = df['error']

seqs= results_new['sequence'][:-2].values

res = results_new['c: 1.8; k: 0.2,0.3,1.6; xyz'][:-2].values
median = [np.median(res)] * len(res)

x = [x for x in range (0,len(seqs))]

# plot CPT baseline
plt.scatter(x, cpt, marker='o', s=20, c='dodgerblue', alpha=0.3, 
            label=str('CPT baseline median: ') + str(round(np.median(cpt),2)))
plt.plot(x, [np.median(cpt)] * len(x), '-', c='dodgerblue', alpha=0.8)
#plt.plot((x_original,x_original),(res_original,distances), '--', c='darkgreen', alpha=0.6)
plt.fill_between(x, cpt, alpha=0.3, color='dodgerblue')


# plot RNN baseline
plt.scatter(x, rnn, marker='o', s=20, c='limegreen', alpha=0.5, 
            label=str('RNN baseline median: ') + str(round(np.median(rnn),2)))
plt.plot(x, [np.median(rnn)] * len(x), '-', c='green', alpha=0.8)
#plt.plot((x_original,x_original),(res_original,distances), '--', c='darkgreen', alpha=0.6)
plt.fill_between(x, rnn, alpha=0.3, color='limegreen')

# plot random baseline
plt.scatter(x, error, c='red', s=20, marker='o', alpha=0.3, 
            label=str('guessing baseline median: ') + str(round(np.nanmedian(results_new['error']),2)))
plt.plot(x, [np.median(error)] * len(x), c='red', alpha=0.8)#, 
         #label=str('guessing baseline avg.: ' + str(round(np.mean(results_original['error']),2))))
plt.fill_between(x, error, [1.0 * len(error)], color='red', alpha=0.2)

# plot scatter + lines for simulations
plt.scatter(x, res, marker='o', s=20, c='navy', alpha=0.6, 
            label=str('model-generated median: ') + str(round(lowest_median,2)))
#plt.plot(x, res, c='blue', alpha=0.6)
plt.plot(x, median, c='navy', alpha=0.6)


plt.xticks(x, labels=IDs, rotation=90, fontsize=5)
#plt.xticklabels(IDs, rotation=90, fontsize=6)

plt.ylabel('normalized edit distance', fontsize=22)
plt.xlabel('sequence', fontsize=22)
plt.ylim(0.0, 0.9)
plt.title('best model (all), c:1.8; k: 0.2,0.3,1.6; xyz', fontsize=26)

plt.legend(fontsize=20, framealpha=0.8, loc='upper right')

#plt.savefig('plot_median_editdist_individualerrors_diff.png', bbox_inches='tight')
plt.show()

### Prep data for stat analysis

In [151]:
# Compare avg edit distance for xy and xyz
list_xy = []
list_xyz = []

for col in results_mean:
    if col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'xy':
        list_xy.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'xyz':
        list_xyz.append(results_mean.at['mean',col])
        
avg_xy = np.mean(list_xy)
med_xy = np.median(list_xy)
std_xy = np.std(list_xy)
print('Average xy: ' + str(avg_xy)  + ', stdev: ' + str(std_xy) + ', median: ' + str(med_xy))
        
avg_xyz = np.mean(list_xyz)
med_xyz = np.median(list_xyz)
std_xyz = np.std(list_xyz)
print('Average xyz: ' + str(avg_xyz) + ', stdev: ' + str(std_xyz) + ', median: ' + str(med_xyz))

Average xy: 0.14382978638113403, stdev: 0.006246028746152526, median: 0.14036504472035805
Average xyz: 0.14307780009927065, stdev: 0.006425768841590568, median: 0.1395177695369387


In [152]:
# Compare avg edit distances for x, y, z
list_x = []
list_y = []
list_z = []
list_xz = []
list_yz = []

for col in results_mean:
    if col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'x':
        list_x.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'y':
        list_y.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'z':
        list_z.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'xz':
        list_xz.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'yz':
        list_yz.append(results_mean.at['mean',col])

avg_xz = np.mean(list_xz)
avg_yz = np.mean(list_yz)
        
avg_x = np.mean(list_x)
lowest_x = min(list_x)
print('Average x: ' + str(avg_x) + ', min: ' + str(lowest_x)) 
        
avg_y = np.mean(list_y)
lowest_y = min(list_y)
print('Average y: ' + str(avg_y) + ', min: ' + str(lowest_y)) 
        
avg_z = np.mean(list_z)
lowest_z = min(list_z)
print('Average z: ' + str(avg_z) + ', min: ' + str(lowest_z)) 

print('Average xz: ' + str(avg_xz)) 
print('Average yz: ' + str(avg_yz)) 

Average x: 0.16380723404694258, min: 0.1584050345414325
Average y: 0.15687516172294916, min: 0.1487791892136648
Average z: 0.16623977810070703, min: 0.15877037964128035
Average xz: 0.1563376829690196
Average yz: 0.1555665467817485


In [153]:
# Compare all error predictions for dimensions
stat, p = friedmanchisquare(list_x,list_y,list_z,list_xy,list_xz,list_yz,list_xyz)
print('Friedman: stat = %.3f, p = % 10.3E' % (stat, p))

Friedman: stat = 3493.810, p =  0.000E+00


In [154]:
# 2D vs 3D median
stat, p = wilcoxon(list_xy, list_xyz, zero_method='wilcox')
print('Wilcoxon: W = %.3f, p = %.5f' % (stat, p))

Wilcoxon: W = 82197.000, p = 0.00000
